```
██╗    ██╗ █████╗ ███╗   ███╗███╗   ███╗██╗   ██╗██╗   ██╗
██║    ██║██╔══██╗████╗ ████║████╗ ████║██║   ██║██║   ██║
██║ █╗ ██║███████║██╔████╔██║██╔████╔██║██║   ██║██║   ██║
██║███╗██║██╔══██║██║╚██╔╝██║██║╚██╔╝██║██║   ██║██║   ██║
╚███╔███╔╝██║  ██║██║ ╚═╝ ██║██║ ╚═╝ ██║╚██████╔╝╚██████╔╝
 ╚══╝╚══╝ ╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚═╝ ╚═════╝  ╚═════╝ 

```
Create a Minecraft Server on Google Colab!

The script below will initiate your server. First, you will need to create the server to be able to use it - don't worry, the scripts below will do the majority of the work for you.

In [ ]:
from google.colab import drive
import os
import subprocess
import requests  # <-- Fix: Import requests

drive.mount('/content/drive')

!apt update
!apt install openjdk-21-jdk -y

server_path = '/content/drive/MyDrive/MinecraftServer'
!mkdir -p {server_path}
%cd {server_path}

server_urls = {
    "vanilla": "https://piston-meta.mojang.com/mc/game/version_manifest.json",
    "paper": "https://api.papermc.io/v2/projects/paper",
    "spigot": "https://download.getbukkit.org/spigot/",
    "forge": "https://maven.minecraftforge.net/net/minecraftforge/forge/"
}

def get_paper_latest_build(version):
    url = f"{server_urls['paper']}/versions/{version}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        builds = data.get("builds", [])
        if builds:
            return builds[-1]  # Latest build
    return None

def get_versions(server_type):
    if server_type == "paper":
        response = requests.get(server_urls["paper"])
        if response.status_code == 200:
            data = response.json()
            return data.get("versions", [])
    elif server_type == "vanilla":
        response = requests.get(server_urls["vanilla"])
        if response.status_code == 200:
            data = response.json()
            return [v["id"] for v in data.get("versions", []) if v["type"] == "release"]
    elif server_type == "forge":
        return ["1.20.1", "1.19.4", "1.18.2"]  # Add more versions manually or scrape a reliable source
    return []

jar_path = f"{server_path}/server.jar"
if not os.path.exists(jar_path):
    while True:
        server_type = input("Enter server type (vanilla/paper/spigot/forge): ").strip().lower()
        if server_type in server_urls:
            break
        print("Invalid server type. Please enter one of: vanilla, paper, spigot, forge.")
    
    versions = get_versions(server_type)
    if versions:
        print("Available versions:", ", ".join(versions[-5:]))  # Show last 5 versions
        selected_version = input("Enter the Minecraft version: ").strip()
        if selected_version not in versions:
            print("Invalid version selected.")
            exit()
    else:
        print("Failed to fetch available versions.")
        exit()

    if server_type == "paper":
        latest_build = get_paper_latest_build(selected_version)
        if latest_build:
            jar_url = f"{server_urls['paper']}/versions/{selected_version}/builds/{latest_build}/downloads/paper-{selected_version}-{latest_build}.jar"
        else:
            print("Failed to fetch the latest build.")
            exit()
    elif server_type == "vanilla":
        jar_url = f"https://piston-data.mojang.com/v1/objects/{selected_version}/server.jar"
    elif server_type == "spigot":
        jar_url = f"{server_urls['spigot']}spigot-{selected_version}.jar"
    elif server_type == "forge":
        jar_url = f"{server_urls['forge']}{selected_version}/forge-{selected_version}-installer.jar"  # Fix URL format

    print("Downloading server JAR...")
    subprocess.run(["wget", "-O", jar_path, jar_url])
else:
    print("Server JAR already exists. Skipping download.")

eula_file = f'{server_path}/eula.txt'
if not os.path.exists(eula_file):
    with open(eula_file, 'w') as f:
        f.write('eula=true')

def is_server_running():
    result = os.popen("ps aux | grep '[j]ava' | grep server.jar").read()
    return bool(result)

def stop_server():
    if is_server_running():
        print("Stopping Minecraft server...")
        os.system("pkill -f 'java.*server.jar'")
    else:
        print("Server is not running.")

while True:
    action = input("Enter 'start' to launch the server or 'stop' to terminate it: ").strip().lower()
    if action in ["start", "stop"]:
        break
    print("Invalid input. Please enter 'start' or 'stop'.")

if action == "start":
    if is_server_running():
        print("Server is already running!")
    else:
        print("Starting Minecraft server...")
        process = subprocess.Popen(f"java -Xmx8G -Xms8G -jar {jar_path} nogui", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        try:
            for line in iter(process.stdout.readline, ""):
                print(line, end="")
        except KeyboardInterrupt:
            print("\nServer stopped manually.")
            stop_server()

elif action == "stop":
    stop_server()
